In [2]:
import polars as pl
from scipy.stats import chi2_contingency
from pathlib import Path

data_dir = Path("../data")

transactions = pl.scan_parquet(data_dir / "transactions_train_3.parquet")


Die Spalte customer_feedback enthält nur in 7.6% der Fälle einen Wert. Der Mittelwert ist mit 9.3 aussergewöhnlich hoch, schon das 25%Quantil liegt bei 10.0. Es ist daher fraglich ob die Spalte für die Analyse nützlich ist.

In [10]:
feedback = transactions.select("customer_feedback").describe()

non_missing_values_in_percent = feedback[0,1] / feedback[1,1]
print(f"Non-missing values in percent: {non_missing_values_in_percent:.2%}")

feedback

Non-missing values in percent: 7.60%


statistic,customer_feedback
str,f64
"""count""",104720.0
"""null_count""",1.377063e6
"""mean""",9.319385
"""std""",1.713751
"""min""",1.0
"""25%""",10.0
"""50%""",10.0
"""75%""",10.0
"""max""",10.0


Ein Chi-Quadrat-Test zeigt aber, dass zumindest das Vorhandensein eines Wertes in der Spalte für unsere Analyse relevant ist. Der p-Wert ist mit 0.001 deutlich kleiner als 0.05. Wir können also die Nullhypothese ablehnen und annehmen, dass es einen Zusammenhang zwischen dem Vorhandensein von customer_feedback und einem Betrugsfall gibt.

In [11]:
transactions_labeled = transactions.filter(pl.col("label") != "UNKNOWN").select(
    (
        pl.col("customer_feedback").is_not_null().cast(pl.Int8).alias("has_feedback"),
        pl.col("label"),
    )
)

col1 = "has_feedback"
col2 = "label"

contingency_table = (
    transactions_labeled
    .group_by(col1, col2)
    .agg(pl.len().alias("count"))
    .collect()
    .pivot(values="count", index=col1, on=col2)
)
contingency_table.to_numpy()
chi2, p, dof, exp = chi2_contingency(contingency_table)
exp = pl.DataFrame(exp, schema=contingency_table.columns)

print(f"Contingency table:\n{contingency_table}")
print(f"Expected frequencies:\n{exp}")
print(f"Chi-squared test result: chi2={chi2}, p-value={p}, dof={dof}")

Contingency table:
shape: (2, 3)
┌──────────────┬───────┬────────┐
│ has_feedback ┆ FRAUD ┆ NORMAL │
│ ---          ┆ ---   ┆ ---    │
│ i8           ┆ u32   ┆ u32    │
╞══════════════╪═══════╪════════╡
│ 0            ┆ 4307  ┆ 133074 │
│ 1            ┆ 349   ┆ 10295  │
└──────────────┴───────┴────────┘
Expected frequencies:
shape: (2, 3)
┌──────────────┬─────────────┬───────────────┐
│ has_feedback ┆ FRAUD       ┆ NORMAL        │
│ ---          ┆ ---         ┆ ---           │
│ f64          ┆ f64         ┆ f64           │
╞══════════════╪═════════════╪═══════════════╡
│ 0.928087     ┆ 4321.172875 ┆ 133058.899038 │
│ 0.071913     ┆ 334.827125  ┆ 10310.100962  │
└──────────────┴─────────────┴───────────────┘
Chi-squared test result: chi2=13.575923099027321, p-value=0.0011272643061889648, dof=2
